In [7]:
import numpy as np
import optunity
import optunity.metrics

# short form for now
original_data = np.genfromtxt('../../working_data/updrsii_short_form.csv', delimiter=',', skip_header=True)
_, n_columns = original_data.shape

data = original_data[:,0:(n_columns - 1)]
labels = original_data[:,(n_columns - 1)]

# Common cross validator for all models
cv_decorator = optunity.cross_validated(x=data, y=labels, num_folds=10)

results = []


In [6]:
# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

def gnb_tuned_auroc(x_train, y_train, x_test, y_test, sigfall_prior):
    no_fall_prior = 1.0 - sigfall_prior
    model = GaussianNB(priors=[no_fall_prior,sigfall_prior]).fit(x_train, y_train)
    decision_values = model.predict(x_test)
    auc = optunity.metrics.roc_auc(y_test, decision_values)
    return auc
    
    
gnb_tuned_auroc = cv_decorator(gnb_tuned_auroc)
gnb_optimal_pars, gnb_info, _ = optunity.maximize(gnb_tuned_auroc, solver_name='grid search', num_evals=100, sigfall_prior=[0.01,0.99])

print("Optimal parameters" + str(gnb_optimal_pars))
print("AUROC of tuned model: %1.3f" % gnb_info.optimum)

results.append({'model': 'Gaussian Naive Bayes',
               'Optimal parameters': gnb_optimal_pars,
               'ROC_AUC': gnb_info.optimum
               })

Optimal parameters{'sigfall_prior': 0.024700000000000024}
AUROC of tuned model: 0.637
